## Read the saved model and use the new data set (not exposed to training) 
## And makes the prediction
## Model trained on pion (+), HCAL, 17 deg, discrete energy
## FUNCTIONS NEEDED
* generator.py
* deepsets_test_prediction.py
* configs/default.yaml

## Instructions how to run the codes 

* You can run the model on continuous or discrete data set remember they may have different mean and std
* I would recommend to copy "means.p" and "std.p" files from respective directory to your "output_dir"
* From output directory 'generator.py" script which produces test dataset will read means and stds 
* For continuous energy means.p and std.p are in directory "/media/miguel/Elements/Data_hcali/Data1/log10_Uniform_03-23/DeepSets_output/preprocessed_data"
* For discrete energy mean.p and std.p are in directory "/media/miguel/Elements/Data_hcali/Data1/log10_Uniform_03-23/DeepSets_output/preprocessed_data_discrete_pi+_17deg_all"

## Not necessary to change anything in cell below
## May be some WARNING from tensorflow

In [1]:
from deepsets_test_prediction import *

data_config = config['data']
model_config = config['model']
train_config = config['training']

#data_dir = data_config['data_dir']                                                                                                                                                         

#num_val_files = data_config['num_val_files']                                                                                                                                               
batch_size = data_config['batch_size']
shuffle = data_config['shuffle']
num_procs = 2
preprocess = data_config['preprocess']

already_preprocessed = False
calc_stats = False
concat_input = model_config['concat_input']

2023-04-06 16:42:37.007596: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 16:42:37.561844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bishnu/Library/root/lib:/usr/local/lib:/usr/local/lib:/usr/local/cuda-11.8/lib64
2023-04-06 16:42:37.561905: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bishnu/Library/root/lib:/usr/local/li

## You can play with these parameters
* CHOOSE THE NUMBER OF FILES FOR THE TEST DATA 
* CHOSE THE DATA 'discrete' OR 'continuous'

In [2]:
## CHOOSE HOW MANY FILES YOU WANT TO USE FOR THE TEST DATA SETS
num_test_files =1

## CHOOSE THE DISCRETE OR CONTINUOUS DATA SET
data_type='discrete'

## CHOOSE THE MODEL DIRECTORY
* path_to_model can remain same (You can read the model from my directory or copy to your place
* path to data can also remain same 
* BUT OUTPUT_DIR NEEDS TO BE AT YOUR PLACE, YOU NEED TO COPY RIGHT MEAN AND STD FILES FROM ABOVE MENTIONED DIRECTORIES TO HERE

In [3]:
## PATH TO THE DATA AND MODEL DIRECTORY                                                                                                                                                     
path_to_model='/media/miguel/Elements/Data_hcali/Data1/log10_Uniform_03-23/DeepSets_output/\
    results_discrete_pi+_17deg_all/Block_20230331_1758_concatTrue'

## TEST DATA SET DIRECTORU
if data_type=='continuous':
    data_dir="/media/miguel/Elements/Data_hcali/Data1/log10_Uniform_03-23/log10_pi+_discrete_17deg_20_20k/"

## FOR CONTINUOUS DATA SET
elif data_type=='discrete':
    data_dir="/media/miguel/Elements/Data_hcali/Data1/log10_Uniform_03-23/log10_pi+_Uniform_0-140Gev_17deg_testdata/"
else:
    print("What is the data type you want to pick")

### WHERE YOU WANT TO PUT THE PREDICTION USING MODEL AND TEST DATA SETS
## This directory should contain the mean and standard deviation
output_dir ='/home/bishnu/EIC/deepsets_output/test_test'


## READ MODEL 
## NO ANY CHANGES NEEDED IN THE CELL BELOW

In [4]:
if preprocess:
        train_output_dir = output_dir + '/train/'
        val_output_dir = output_dir + '/test/'

### MODEL READ                                                                                                                                                                              
model = models.BlockModel(global_output_size=1, model_config=model_config)
checkpoint = tf.train.Checkpoint(module=model)

last_ckpt_path = path_to_model + '/last_saved_model'

if os.path.exists(last_ckpt_path+'.index'):
        checkpoint.read(last_ckpt_path)
        
        
root_files = np.sort(glob.glob(data_dir+'*root'))
test_start = 0
test_end = test_start + num_test_files
#val_end = train_end + num_val_files                                                                                                                                                        
root_test_files = root_files[test_start:test_end]        

## GENERATES THE TEST DATA SET

In [5]:
data_gen_val = MPGraphDataGenerator(file_list=root_test_files,
                                        batch_size=batch_size,
                                        shuffle=shuffle,
                                        num_procs=num_procs,
                                        calc_stats=calc_stats,
                                        is_val=True,
                                        preprocess=preprocess,
                                        already_preprocessed=already_preprocessed,
                                        output_dir=val_output_dir)
                                      


Preprocessing and saving data to /home/bishnu/EIC/deepsets_output/test_test/test
Processing file number 0
Finished processing file number 0


In [6]:
samp_graph, samp_target = next(get_batch(data_gen_val.generator()))
data_gen_val.kill_procs()
graph_spec = utils_tf.specs_from_graphs_tuple(samp_graph, True, True, True)

2023-04-06 16:43:00.085428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-06 16:43:00.085763: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-06 16:43:00.105334: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bishnu/Library/root/lib:/usr/local/lib:/usr/local/lib:/usr/local/cuda-11.8/lib64
2023-04-06 16:43:00.105361: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed proper

## MAKES THE MODEL PREDICTION AND SAVES THE PREDICTION 

In [7]:
all_targets = []
all_outputs = []

start = time.time()

for graph_data_val, targets_val in get_batch(data_gen_val.generator()):#val_iter):                                                                                                          
        #losses_val, output_vals = val_step(graph_data_val, targets_val)                                                                                                                    
        #print('i -s ********   ', i)                                                                                                                                                       
        output_vals = val_step(graph_data_val, targets_val)
        
        targets_val = targets_val.numpy()
        output_vals = output_vals.numpy().squeeze()

        all_targets.append(targets_val)
        all_outputs.append(output_vals)
        
all_targets = np.concatenate(all_targets)
all_outputs = np.concatenate(all_outputs)


np.savez(output_dir+'/predictions_appended', targets=all_targets, outputs=all_outputs)

print("HELLO I  AM DONE")

HELLO AI M DONE
